# Нейронка на gpt-3 для векторизации описания фильмов

# import

In [4]:
from Database.Database import DataFunFilm
import pandas as pd

# Загрузка базы данных

In [5]:
db = DataFunFilm()
db.getTableDF()

,id,kinopoisk_Id,imdb_id,name_ru,name_original,poster_url,rating_imdb,rating_imdb_vote_count,rating_film_critics,rating_critics_vote_count,year,film_length,genres,web_url,slogan,description
0,1,300,tt0306841,Лиззи Магуайр,The Lizzie McGuire Movie,https://kinopoiskapiunofficial.tech/images/pos...,5.5,39148,5.3,102,2003.0,94.0,мелодрама приключения комедия музыка семейный,https://www.kinopoisk.ru/film/300/,Lizze McGuire goes to Rome.,Тринадцатилетняя школьница Лиззи Магуайер и ее...
1,2,301,tt0133093,Матрица,The Matrix,https://kinopoiskapiunofficial.tech/images/pos...,8.7,1839106,7.8,158,1999.0,136.0,фантастика боевик,https://www.kinopoisk.ru/film/301/,Добро пожаловать в реальный мир,Жизнь Томаса Андерсона разделена на две части:...
2,3,302,tt0311289,Клад,Holes,https://kinopoiskapiunofficial.tech/images/pos...,7.0,85487,7.0,138,2003.0,117.0,драма детектив приключения комедия семейный,https://www.kinopoisk.ru/film/302/,"Кто ищет, тот всегда найдёт",Стэнли арестован по ложному обвинению в краже ...
3,4,303,tt0305224,Управление гневом,Anger Management,https://kinopoiskapiunofficial.tech/images/pos...,6.2,212256,5.1,193,2003.0,106.0,мелодрама комедия,https://www.kinopoisk.ru/film/303/,Let the healing begin,Скромному клерку отчаянно не везет. Парня по о...
4,5,304,tt0310910,Афера,Confidence,https://kinopoiskapiunofficial.tech/images/pos...,6.6,34131,6.3,157,2003.0,97.0,триллер криминал,https://www.kinopoisk.ru/film/304/,"Это не про деньги, это про ДЕНЬГИ!!!",Джейк Вига – хитроумный и обаятельный мошенник...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70300,70301,219369,tt0071859,Путешественник,Mossafer,https://kinopoiskapiunofficial.tech/images/pos...,7.5,1695,NaN,0,1974.0,83.0,драма,https://www.kinopoisk.ru/film/219369/,None,Парнишке-школьнику по имени Кассем из маленько...
70301,70302,47614,tt0211372,Сын француза,Le fils du Français,https://kinopoiskapiunofficial.tech/images/pos...,5.0,282,NaN,0,1999.0,107.0,приключения комедия семейный,https://www.kinopoisk.ru/film/47614/,None,У Бенжамена 9 лет нет больше мамы. Он хочет на...
70302,70303,412287,tt1226214,Приключения молодого Индианы Джонса: Шпионские...,The Adventures of Young Indiana Jones: Espiona...,https://kinopoiskapiunofficial.tech/images/pos...,6.8,248,NaN,0,2000.0,92.0,приключения комедия,https://www.kinopoisk.ru/film/412287/,None,None
70303,70304,977755,tt9378362,Звоните ДиКаприо!,None,https://kinopoiskapiunofficial.tech/images/pos...,6.7,496,NaN,0,2018.0,48.0,драма комедия,https://www.kinopoisk.ru/series/977755/,Что останется после тебя?,Егор Румянцев – звезда популярного сериала про...
